 ╔══<i><b>Alai-DeepLearning</b></i>═══════════════════════════╗
###  &nbsp;&nbsp; **✎&nbsp;&nbsp;week 4. Tensorflow basis**
# Section 7. Tensorflow 실습 해보기 

### _Objective_
1.  Fractal 의 일종인 mandelbrot 을 생성해 봅니다. 
2. Fractal 의 일종인 julia을 생성해 봅니다. 

╚════════════════════════════════════════╝

In [ ]:
%matplotlib inline
!pip install tensorboardcolab

import tensorboardcolab
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

# \[ 1. Mandelbrot 만들어 보기  \]

---

---

> fractal 은 복소수 수식을 통해 생성되며 큰 도형의 일부가 새롭게 생성된 작은 도형의 모형과 비례합니다.

## 1.  그래프를 보고 따라 만들어보기 

---

+ ppt 을 보고 Graph 을 구현해보세요 .<br>


### (1)  mesh grid 을 이용해 coordinate 만들어 보기 
mgrid 또는 mesh grid 을 사용하는 법을 배워 봅니다. 

In [ ]:
Y,X = np.mgrid[-1.3: 1.3 : 0.005 , -2:1:0.005] 
init_value = X +1j*Y

In [ ]:
xs=tf.constant(init_value,name='xs')
print(xs)

### (2)  필요한 변수, 연산 들을 정의

In [ ]:
zs = tf.Variable(xs , name='zs')
zs_square = tf.multiply(zs,zs ,name='zs_square')
zs_add = tf.add(zs_square , xs)

zs_abs = tf.abs(zs_add , name='zs_abs')
zs_less = tf.math.less(zs_abs, 4 , name='zs_less')
zs_cast = tf.cast(zs_less , tf.float32)

### (3)   결과값이 저장되어 이미지화 되는 변수(ns) 을 생성

In [ ]:
xs_zeros = tf.zeros_like(xs , dtype=tf.float32 , name='xs_zeros')
ns = tf.Variable(xs_zeros , name='ns')

### (4)  tf.group 이용해서 한번에 assign 시키기

In [ ]:
step = tf.group( tf.assign(zs , zs_add) , tf.assign_add(ns , zs_cast))

### (5) 학습 해보기

In [ ]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for i in range(200):
    sess.run(step) # step 을 한번 실행 
result = sess.run(ns)

### (6) 시각화 해보기 

In [ ]:
plt.imshow(result)
plt.show()

## 2.  Madelbrot을 텐서 보드에 연결하기 
---

+ Mandelbrot 을 학습시키고 그 과정을 Tensorboard 와 연결시킵니다

In [ ]:
tf.reset_default_graph()

# add tensorboard
# input initial value
Y, X = np.mgrid[-1.3:1.3:0.005, -2:1:0.005]
init_value = X + 1j*Y

with tf.name_scope('cal'):
    xs = tf.constant(init_value )
    zs = tf.Variable(xs)
    zs_zeros = tf.zeros_like(xs, tf.float32)
    ns = tf.Variable(zs_zeros,name='ns')

# add tensor to tensorboard
ns_image_tb = tf.summary.image(name='ns_image',
                               tensor=tf.reshape(ns,shape=[1,520,600,1]))
ns_mean_tb = tf.summary.scalar(name='ns_mean',
                               tensor=tf.reduce_mean(ns))
ns_hist_tb = tf.summary.histogram(name='ns_hist',
                                 values=ns)

zs_square = tf.multiply(zs,zs)
zs_add = tf.add(zs_square , xs)
zs_abs = tf.abs(zs_add)
zs_less = tf.math.less(zs_abs , 4)
zs_cast = tf.cast(zs_less , tf.float32)

step = tf.group(
  tf.assign(zs, zs_add),
  tf.assign_add(ns, zs_cast),
  name = 'step'
)

sess = tf.Session()
sess.run(tf.global_variables_initializer())
saver = tf.train.Saver()
tbs = tf.summary.merge_all()

tbc=tensorboardcolab.TensorBoardColab(graph_path='./mandelbrot')
writer = tf.summary.FileWriter(logdir = './mandelbrot')
writer.add_graph(tf.get_default_graph())

for i in range(200):
    _, tbs_ = sess.run([step,tbs])
    writer.add_summary(tbs_, global_step=i)
    
writer.flush()

saver.save(sess,
           save_path='./model/mandelbrot')
value = sess.run(ns)
plt.imshow(value)
plt.show()

## 3.  Madelbrot을 복원하기
---

+ 그래프와 저장한 Mandelbrot 변수를 불러옵니다 

In [ ]:
tf.reset_default_graph()
saver = tf.train.import_meta_graph('./model/mandelbrot.meta')
sess = tf.Session()
saver.restore(sess, './model/mandelbrot')
ns = tf.get_default_graph().get_tensor_by_name('cal/ns:0')
step = tf.get_default_graph().get_operation_by_name('step')
sess.run('step')
ns_ = sess.run(ns)

plt.imshow(ns_)
plt.show()

# \[ 2.Julia  집합 만들어 보기 \]

---

---

> fractal 의 다른 종류인 Julia 집합을 만들어 봅니다. 


### (1)  mesh grid 을 이용해 coordinate 만들어 보기 

In [ ]:
Y,X = np.mgrid[-2:2:0.005 , -2:2:0.005]
init_value = X + 1j*Y

### (2)  필요한 변수, 연산 들을 정의

In [ ]:
xs = tf.constant(init_value , name='xs')
zs = tf.Variable(xs)
zs_square = zs * zs
zs_sub = tf.subtract(zs_square , 0+0.75j , name='zs_sub')

zs_abs = tf.abs(zs_sub , name='zs_abs')
zs_less = tf.math.less(zs_abs, 4, name='zs_less')
zs_cast = tf.cast(zs_less , tf.float32 , name='zs_cast')

### (3)   결과값이 저장되어 이미지화 되는 변수(ns) 을 생성

In [ ]:
xs_zeros = tf.zeros_like(xs , dtype=tf.float32, name='xs_zeros')
ns = tf.Variable(xs_zeros , name='ns')

### (4)  tf.group 이용해서 한번에 assign 시키기

In [ ]:
step = tf.group(tf.assign(zs , zs_sub) , tf.assign_add(ns , zs_cast))

### (5) 학습 해보기

In [ ]:
# 변수 초기화 
sess = tf.Session()
init_op = tf.global_variables_initializer()
sess.run(init_op)

for i in range(200):
    sess.run(step)
result = sess.run(ns)

### (6) 시각화 해보기

In [ ]:
plt.imshow(result)
plt.show()

#  

---

    Copyright(c) 2019 by Public AI. All rights reserved.
    Writen by PAI, SangJae Kang ( rocketgrowthsj@publicai.co.kr )  last updated on 2019/03/22


---